In [1]:
pip install openai wikipedia-api serpapi requests

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=c850b4fd43b3d959fc1778f31c354796396c9baaa00d7078663cb5532f98aec4
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [2]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [3]:
pip install gtts

In [9]:
import openai
import requests
import wikipediaapi
import io
import time
from PIL import Image
from moviepy.editor import *  # For video editing
from gtts import gTTS  # For text-to-speech

# ✅ OpenAI API Key (Ensure you set this in your environment variables)
OPENAI_API_KEY = "*********"

# ✅ Initialize OpenAI Client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ Wikipedia API Setup
wiki = wikipediaapi.Wikipedia(user_agent="MyTravelApp/1.0", language="en")

# ✅ Function to fetch travel data from Wikipedia
def get_wikipedia_summary(place):
    page = wiki.page(place)
    return page.summary[:500] if page.exists() else "No Wikipedia summary found."

# ✅ Function to generate travel story using GPT-4
def generate_travel_story(destination):
    wikipedia_info = get_wikipedia_summary(destination)
    prompt = f"""
    Create an engaging travel story that user is gonna experience when they visit {destination}, highlighting:
    - Must-visit places (historical, cultural, and modern attractions)
    - Best street foods and cafes
    - Unique cultural experiences

    Wikipedia Summary: {wikipedia_info}

    Story Output:
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a travel storyteller."},
                  {"role": "user", "content": prompt}],
        max_tokens=500
    )

    return response.choices[0].message.content.strip()

# ✅ Function to generate multiple images using OpenAI DALL-E
def generate_travel_images(destination):
    """Generate a realistic animated set of images representing the travel experience"""
    prompts = [
        f"A breathtaking aerial view of {destination}, sunset, ultra-HD",
        f"A vibrant street food market in {destination}, full of people enjoying food",
        f"A cozy and famous café in {destination}, warm atmosphere, travelers enjoying coffee",
        f"A historic landmark in {destination}, cinematic lighting, tourists exploring",
        f"A scenic park or nature spot near {destination}, peaceful, relaxing vibes"
    ]

    image_urls = []
    for prompt in prompts:
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            n=1,
            size="1024x1024"
        )
        image_urls.append(response.data[0].url)

    return image_urls[:5]  # ✅ Limiting to 5 images

# ✅ Function to create a voice-over for the travel story
def generate_voiceover(story_text, output_audio="travel_narration.mp3"):
    tts = gTTS(text=story_text, lang="en", slow=False)
    tts.save(output_audio)
    return output_audio

# ✅ Function to create an animated travel video with multiple images and synchronized narration
def create_travel_video(image_urls, narration_audio, output_video="travel_story.mp4"):
    """Creates an animated travel video with zoom effects and background narration"""

    # ✅ Load narration audio and calculate duration
    audio_clip = AudioFileClip(narration_audio)
    total_audio_duration = audio_clip.duration  # Total narration time

    # ✅ Adjust image duration to match audio duration
    num_images = len(image_urls)
    image_duration = total_audio_duration / num_images  # Equal time per image

    image_clips = []
    for i, url in enumerate(image_urls):
        # Download the image
        response = requests.get(url)
        image = Image.open(io.BytesIO(response.content))
        image_path = f"travel_image_{i}.jpg"
        image.save(image_path)

        # Load image into MoviePy and add zoom effect
        clip = ImageClip(image_path, duration=image_duration).set_fps(24)
        clip = clip.resize(lambda t: 1 + 0.01 * t)  # Slow zoom-in effect
        image_clips.append(clip)

    # ✅ Combine all images into a single video
    video_clip = concatenate_videoclips(image_clips, method="compose")

    # ✅ Set audio to match video length
    video_clip = video_clip.set_audio(audio_clip)

    # ✅ Export final video
    video_clip.write_videofile(output_video, codec="libx264", fps=24, audio_codec="aac")
    print("🎬 Travel story animation created successfully!")

# ------------------- EXECUTION -------------------

# 🎯 Example Destination
destination = "Hyderabad"

# ✅ Generate Travel Story
story = generate_travel_story(destination)
print("Generated Travel Story:\n", story)

# ✅ Generate Travel Images
image_urls = generate_travel_images(destination)

# ✅ Generate Voice-over for the Story
narration_audio = generate_voiceover(story)

# ✅ Create Animated Travel Video with Narration
create_travel_video(image_urls, narration_audio)


Generated Travel Story:
 Welcome, dear adventurers, to the grand city of Hyderabad, the Pearl City of India. Flanked with centuries of vibrant history, rich Nizami culture, and modern allure, this city will surely leave imprints in the memory of every traveler that passes its threshold.

Your first morning here pours in with the luminescent golden sunbeam bouncing off the majestic Charminar, an iconic symbol of Hyderabad. This iconic structure stands tall in the heart of the city, reaching for the azure sprawl above, surrounded by the bustling, vibrant markets of Laad Bazar. Tag along with the locals haggling for a good bargain for bangles, pearls, and traditional attire, imbuing the atmosphere with an age-old charm. 

A few blocks away, silence and history whisper tales in the archaic walls of the Golconda Fort. Feel the echoes of eons resounding through the ruins, unveiling tales of victories, defeats and dynasties that once harbored in its heart. By nightfall, this fortress lightens

MoviePy - Done.
Moviepy - Writing video travel_story.mp4



Moviepy - Done !
Moviepy - video ready travel_story.mp4
🎬 Travel story animation created successfully!
